In [ ]:
import os
import glob
import trackpy as tp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.optimize import curve_fit
import numpy as np
from matplotlib.ticker import ScalarFormatter
from utils import *

framerate = 4
microns_per_pixel = 1  # I think
px_to_nm = 1000
frame_to_s = 0.25

In [ ]:
out_dir = r"C:\Users\lizau\Desktop\walker_tracker_for_article\walker_tracker\example\simulated_random_walk"

In [ ]:
def random_walk(traj_len, time, sample_size):
    #trajectory_df = pandas.DataFrame()
    frames = np.linspace (0., time, traj_len)
    dt = frames[1] - frames[0]
    dB = np.sqrt(dt)*np.random.normal(size=(traj_len-1, sample_size))
    B0 = np.zeros(shape=(1, sample_size))
    loc = np.concatenate((B0, np.cumsum(dB, axis=0)), axis=0)
    plt.plot(frames, loc)
    plt.show()
    data = {
    'frame': np.repeat(np.arange(traj_len), sample_size),
    'y': loc.flatten(),
    'particle': np.tile(np.arange(sample_size), traj_len)
    }
    trajectory_df = pd.DataFrame(data)
    return trajectory_df

def random_walk_2d_anisotropic(traj_len, time, sample_size, anisotropy_factor=2):
    frames = np.linspace(0., time, traj_len)
    dt = frames[1] - frames[0]
    print(dt)
    # Simulate x and y displacements (independent)
    dB_x = np.sqrt(dt) * np.random.normal(size=(traj_len - 1, sample_size))
    dB_y = anisotropy_factor * np.sqrt(dt) * np.random.normal(size=(traj_len - 1, sample_size))  # 2x larger steps in y

    B0_x = np.zeros((1, sample_size))
    B0_y = np.zeros((1, sample_size))

    # Cumulative sum for positions
    loc_x = np.concatenate((B0_x, np.cumsum(dB_x, axis=0)), axis=0)
    loc_y = np.concatenate((B0_y, np.cumsum(dB_y, axis=0)), axis=0)

    # Plot the trajectory of the first few particles
    plt.figure(figsize=(6, 6))
    for i in range(sample_size):
        plt.plot(loc_x[:, i], loc_y[:, i], alpha=0.6, label=f'Particle {i}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('2D Anisotropic Random Walks (Y steps 2x larger)')
    plt.grid(True)
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

    # Create DataFrame
    df = pd.DataFrame({
        'frame': np.repeat(np.arange(traj_len), sample_size),
        'x': loc_x.flatten(),
        'y': loc_y.flatten(),
        'particle': np.tile(np.arange(sample_size), traj_len)
    })

    return df


In [ ]:
traylen = 50
T = 12.5
num_particles = 50

data = random_walk_2d_anisotropic(traylen, T, num_particles)
random_walk_data = Path(f'{out_dir}/random_walk_traylen{traylen}_time{T}_num_particles{num_particles}.csv')
data.to_csv()

In [ ]:
# Calculate and plot imsd and emsd for simulated random walk (for comparison with experimental data)

tray_len_query = traylen
max_lag = 50
coordinate = ['x']

formatter = ScalarFormatter()
formatter.set_scientific(True)
formatter.set_powerlimits((-2, 2))  # controls when sci notation kicks in

im = tp.imsd(data, microns_per_pixel, framerate, max_lagtime=max_lag, pos_columns=coordinate)
fig, ax = plt.subplots()
ax.plot(im.index, im, "k-", alpha=0.1)  # black lines, semitransparent
ax.set_ylabel(r"$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]", fontsize=14)
ax.set_xlabel("lag time $t$ [s]", fontsize=14)
yticks = np.linspace(np.round(np.min(im),5), np.round(np.max(im),5), 5)
ax.set_yticks(yticks)
ax.yaxis.get_offset_text().set_fontsize(14)
ax.yaxis.set_major_formatter(formatter)
ax.tick_params(axis='both', labelsize=14)
ax.set_title(f'imsd for trajectories longer than {tray_len_query}')


em = tp.emsd(data, microns_per_pixel, framerate, max_lagtime=max_lag, pos_columns=coordinate)
fig, ax = plt.subplots()
ax.plot(em.index, em, "o")
ax.set_ylabel(r"$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]", fontsize=14)
ax.set_xlabel("lag time $t$ [s]", fontsize=14)
#ax.set_title(f'emsd for trajectories longer than {tray_len_query}')
yticks = np.linspace(np.round(np.min(em),5), np.round(np.max(em),4), 5)
ax.set_yticks(yticks)
ax.yaxis.get_offset_text().set_fontsize(14)
ax.yaxis.set_major_formatter(formatter)
ax.tick_params(axis='both', labelsize=14)

plt.savefig(f'{out_dir}/emsd_simulated_traylen{traylen}_numparticles{num_particles}_{coordinate}.png', bbox_inches='tight')



In [ ]:
# This cell fits a linear funtion to the EMSD graph on log scale
    # The slope gives the parameter alpha

def linear_func(x, m, b):
    return m * x + b

linear_start = 0
linear_end = 50

# Extract the specific part of the data for fitting
mask = (em.index > linear_start) & (em.index < linear_end)
x_linear = em.index[mask]
y_linear = em[mask]


# Fit the linear region with curve_fit
popt, pcov = curve_fit(linear_func, np.log10(x_linear), np.log10(y_linear))
slope, intercept = popt
print(slope)

# Plot the original data and the fitted line
fig, ax = plt.subplots()
ax.plot(em.index, em, "o")
ax.plot(x_linear, 10**intercept*x_linear**slope, label=f"Linear Fit: y = {slope:.2e}x + {np.round(intercept,2)}")
ax.set_ylabel(r"$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]", fontsize=14)
ax.set_xlabel("lag time $t$ [s]", fontsize=14)
#ax.set_title(f"Linear Fit for emsd on log/log scale")
#ax.set_xscale("log")
#ax.set_yscale("log")
yticks = np.linspace(np.round(np.min(em),5), np.round(np.max(em),5), 5)
ax.set_yticks(yticks)
ax.yaxis.get_offset_text().set_fontsize(14)
ax.yaxis.set_major_formatter(formatter)
ax.tick_params(axis='both', labelsize=14)
ax.minorticks_off()
ax.legend(fontsize=12)

plt.savefig(f'{out_dir}/simulated_traylen{traylen}_numparticles{num_particles}_emsd_fit_range_{linear_start}_{linear_end}_{coordinate}.png', bbox_inches='tight')

# Print the fit parameters
print(f"Slope: {slope:.2e}, Intercept: {intercept:.2e}")

In [ ]:
# This cell fits a 2D Gaussian fitting function used for analysis of experimental movies to simulated data
    # Basically a sanity check

steps = data.groupby(["particle"]).apply(get_steps_from_df)
steps["step_len"] = np.sqrt(steps.dx**2 + steps.dy**2)
x = steps['dx']
y = steps['dy']

bins = (75, 75)
#hist, xedges, yedges = np.histogram2d(x, y, bins=(75,50), density=True)
hist, xedges, yedges = np.histogram2d(x, -y, bins=(50,75), density=True)
xcent = (xedges[1:] + xedges[:-1])/2
ycent = (yedges[1:] + yedges[:-1])/2
x_grid, y_grid = np.meshgrid(xcent, ycent)
xy = np.stack((x_grid.ravel(), y_grid.ravel())).T

info = {}

diff_info_out = Path(f'{out_dir}/diff_info_simulated_traylen{50}_numparticles{num_particles}.diff')

initial_guess = [0, 0, 1, 1, 0]
try:
    popt, _ = curve_fit(gaussian_2d, xy, hist.T.ravel(), initial_guess)
except RuntimeError as e:
    warnings.warn(f"Optimal parameters not found: {e}")
    popt = None 
if popt is not None:
    mu = [popt[0], popt[1]]
    cov_matrix = [[popt[2], popt[4]], [popt[4], popt[3]]]
    print(cov_matrix)
    e1, e2, rot = cov_to_axes_and_rotation(cov_matrix)
    print(e1, e2)

if popt is not None:
    #info["diff_long_nm_nm_s"] = float(e1*(px_to_nm**2)/(2*frame_to_s))
    #info["diff_short_nm_nm_s"] = float(e2*(px_to_nm**2)/(2*frame_to_s))
    info["diff_long_nm_nm_s"] = float(e1*px_to_nm*px_to_nm/(2*frame_to_s))
    info["diff_short_nm_nm_s"] = float(e2*px_to_nm*px_to_nm/(2*frame_to_s))
    info["time_per_1000nm_s_long_axis"] = 1000 * 1000 / info["diff_long_nm_nm_s"] / 2
    info["mean_step_nm_s"] = float(np.sqrt(2 * info["diff_long_nm_nm_s"] * frame_to_s) / frame_to_s)
    info["mu_x"] = float(round(mu[0], 3))
    info["mu_y"] = float(round(mu[1], 3))
    info["sigma_x"] = float(round(cov_matrix[0][0], 3))
    info["sigma_y"] = float(round(cov_matrix[1][1], 3))
    info["sigma_xy"] = float(round(cov_matrix[0][1], 3))

    diff_coeff_e1 = [info["diff_long_nm_nm_s"]]
    diff_coeff_e2 = [info["diff_short_nm_nm_s"]]
    time_1000_nm = [info["time_per_1000nm_s_long_axis"]]

    z = gaussian_2d(xy, *popt)
write_yaml(info, diff_info_out)

plt.hist2d(x,-y, bins=bins, density=True, cmap='Blues')
plt.gca().set_aspect('equal', adjustable='box')
if popt is not None:
    plt.contour(x_grid, y_grid, z.reshape(x_grid.shape), cmap='Blues')
    #plt.contour(-y_grid, x_grid, z.reshape(x_grid.shape), cmap='Blues')
plt.colorbar(label='probability density')
plt.xlabel('dx')
plt.ylabel('dy')
plt.title('probability density of step sizes')

plt.savefig(f'{out_dir}/simulated_traylen{traylen}_numparticles{num_particles}_2d_fit.png', bbox_inches = 'tight')
plt.show()

diff_params = {'e1': diff_coeff_e1, 'e2': diff_coeff_e2, 'time_1000_nm': time_1000_nm}
diff_params_df = pd.DataFrame(diff_params)
diff_params_df.to_csv(f'{out_dir}/diff_info.csv')
